## Read the feature selected  data frame via mutual information

In [11]:
import pandas as pd
import numpy as np
import time

np.random.seed(33)
# df.to_csv("../data/cleanedOnlyTriage.csv")
df = pd.read_csv("../data/featureSelectedAllDataWithY.csv")
# df = df.drop("Unnamed: 0", axis=1)
df.head()
# print(df.shape)

,esi,age,ethnicity,race,lang,maritalstatus,employstatus,insurance_status,disposition,arrivalhour_bin,hrtvalvedx,meds_analgesicandantihistaminecombination,meds_antiparkinsondrugs,meds_blood,meds_cardiacdrugs,meds_diagnostic,meds_eentpreps,meds_elect/caloric/h2o,meds_pre-natalvitamins,meds_unclassifieddrugproducts
0,4,40,1,8,1,8,2,4,2,1,0,0,0,0,0,0,0,0,0,0
1,4,66,1,4,1,5,3,1,2,5,0,0,0,0,0,0,0,0,0,0
2,2,66,1,4,1,5,3,1,2,4,0,0,0,0,0,0,0,0,0,0
3,2,66,1,4,1,5,3,1,2,4,0,0,0,0,0,0,0,0,0,0
4,3,84,1,5,2,10,6,3,1,3,0,0,0,0,0,0,0,2,0,0


## Read the feature selected columns for training

In [7]:
## Split the data into training and testing data
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(df, test_size=0.2, random_state=25)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 427784
No. of testing examples: 106946


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif, f_classif

y_train = training_data.iloc[:,-1]
y_test = testing_data.iloc[:,-1]
X_train = training_data.iloc[:,:-1]
X_test = testing_data.iloc[:,:-1]

y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)

start = time.time()
selector = SelectKBest(mutual_info_classif, k=20).fit(X_train, y_train)
cols = selector.get_support(indices=True)
end = time.time()
print(cols) #[ 0  3  5  7  9 10 13 23 24 27 46 48 50 54 60 61 62 64 65 69]
print(end-start)

In [ ]:
X_train_df = pd.DataFrame(X_train)
X_train_smi = X_train_df.iloc[:,cols]
X_test_df = pd.DataFrame(X_test)
X_test_smi = X_test_df.iloc[:,cols]
print(X_train_smi.shape)
print(X_test_smi.shape)

In [ ]:
# print(X_train.shape)
# print(X_test.shape)
# X_test = X_test[:10,]
# y_test = y_test[:10,]

X_train = np.array(X_train)
X_test = np.array(X_test)

# LogisticRegression(random_state=0).fit(X, y)

Logistic Regression

In [ ]:
import pandas as pd
import numpy as np
import time

np.random.seed(33)
df = pd.read_csv("../data/featureSelectedOnlyTriage.csv")
df = df.drop(df.columns[0],axis=1)
# df = df.drop(df.columns[0],axis=1)
df.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(df, test_size=0.2, random_state=25)

y_train = training_data.iloc[:,-1]
y_test = testing_data.iloc[:,-1]
X_train = training_data.iloc[:,:-1]
X_test = testing_data.iloc[:,:-1]

y_train = np.array(y_train)[:100,]
y_test = np.array(y_test)[:10,]
X_train = np.array(X_train)[:100,]
X_test = np.array(X_test)[:10,]
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test,y_test)

SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
sv_clf = SVC(kernel="linear",random_state=1)
sv_clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
sv_clf_pred = sv_clf.predict(X_test)
sv_clf_acc = accuracy_score(y_test,sv_clf_pred)

sc_clf_acc_format = round(sv_clf_acc*100,2)
print("Accuracy of linear SVM model", sc_clf_acc_format)
